In [22]:
import pandas as pd
import requests
import bs4

pd.set_option('display.float_format', '{:,.1f}'.format)

In [53]:
url = 'http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=105560&fin_typ=0&freq_typ=Y'

result=pd.read_html(url, encoding='utf-8')


ValueError: No text parsed from document: http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=105560&fin_typ=0&freq_typ=Y

In [52]:
for i in result:
    print(i)

                                                   0
0  삼성전자  005930  SamsungElec  KOSPI : 전기전자  WICS ...
1  EPS 6,024  BPS 35,342  PER 7.55  업종PER 6.44  P...
                   0                                1
0        주가/전일대비/수익률      45,500원 /  -1,050원 / -2.26%
1      52Weeks 최고/최저               53,900원 /  36,850원
2                액면가                             100원
3           거래량/거래대금            8,699,730주 /  3,944억원
4               시가총액                      2,716,251억원
5              52주베타                             1.52
6         발행주식수/유통비율          5,969,782,550주 / 78.79%
7             외국인지분율                           56.82%
8  수익률 (1M/3M/6M/1Y)  -3.91%/ +17.27%/ -4.01%/ -8.49%
   신용등급  BOND  CP
0   KIS   nan nan
1    KR   nan nan
2  NICE   nan nan
3   NaN   nan nan
                  0
0  [기준: 2019.03.25]
                                                주요주주       보유주식수(보통)  보유지분(%)
0                              삼성물산 외 12인 삼성물산 외 12인 1,266,327,149.0     21.2
1              

In [54]:
import re
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup

'''
get_date_str(s) - 문자열 s 에서 "YYYY/MM" 문자열 추출
'''
def get_date_str(s):
    date_str = ''
    r = re.search("\d{4}/\d{2}", s)
    if r:
        date_str = r.group()
        date_str = date_str.replace('/', '-')

    return date_str

'''
* code: 종목코드
* fin_type = '0': 재무제표 종류 (0: 주재무제표, 1: GAAP개별, 2: GAAP연결, 3: IFRS별도, 4:IFRS연결)
* freq_type = 'Y': 기간 (Y:년, Q:분기)
'''
def get_finstate_naver(code, fin_type='0', freq_type='Y'):
    url_tmpl = 'http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?' \
                   'cmp_cd=%s&fin_typ=%s&freq_typ=%s'

    url = url_tmpl % (code, fin_type, freq_type)
    #print(url)

    dfs = pd.read_html(url, encoding="utf-8")
    df = dfs[0]
    if df.ix[0,0].find('해당 데이터가 존재하지 않습니다') >= 0:
        return None

    df.rename(columns={'주요재무정보':'date'}, inplace=True)
    df.set_index('date', inplace=True)

    cols = list(df.columns)
    if '연간' in cols: cols.remove('연간')
    if '분기' in cols: cols.remove('분기')
    cols = [get_date_str(x) for x in cols]
    df = df.ix[:, :-1]
    df.columns = cols
    dft = df.T
    dft.index = pd.to_datetime(dft.index)

    # remove if index is NaT
    dft = dft[pd.notnull(dft.index)]
    return dft

In [55]:
df = get_finstate_naver('005930')
df[['매출액','영업이익', '당기순이익', '영업활동현금흐름', '순이익률']]

ValueError: No text parsed from document: http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=005930&fin_typ=0&freq_typ=Y